# Deltatorch Example
This notebook briefly demonstrates how to use the [deltatorch](https://github.com/delta-incubator/deltatorch) python library to use `DeltaLake` tables as a data source for model training using PyTorch. The deltatorch library allows users to create a PyTorch `DataLoader` from a DeltaLake dataset.

In this end-to-end example, we'll create a `DataLoader` from a Delta Lake table containing the MNIST dataset, and use it to train a PyTorch CNN model. The `deltatorch` library enables creating PyTorch `DataLoader`s directly from Delta Lake tables, avoiding any conversion to Pandas/NumPy. This allows efficient, scalable data loading from Delta Lake into PyTorch.

## Installation

In [ ]:
%pip install git+https://github.com/delta-incubator/deltatorch

In [ ]:
import pyspark
from delta import *

builder = pyspark.sql.SparkSession.builder.appName("deltatorch-example") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config("spark.executor.memory", "8g")

spark = configure_spark_with_delta_pip(builder).getOrCreate()

## Get some Data
To provide an example with real data, we'll download the `mnist` dataset and save it as a Delta Lake.

In [ ]:
from torchvision import datasets
from pyspark.sql.types import IntegerType, StructType, StructField, FloatType, BinaryType
import numpy as np

In [ ]:
train_set = datasets.MNIST(root='./data', train=True, download=True)
test_set = datasets.MNIST(root='./data', train=False, download=True)

# Convert the data to a Spark DataFrame
schema = StructType([
    StructField("id", IntegerType(), False),
    StructField("label", FloatType(), False),
    StructField("features", BinaryType(), False)  # Changed ArrayType(IntegerType()) to BinaryType()
])

# Convert images to numpy arrays and save as binary
train_data = [(i, float(y), bytearray(np.array(x))) for i, (x, y) in enumerate(train_set)]
train_df = spark.createDataFrame(train_data, schema).repartition(50)

test_data = [(i, float(y), bytearray(np.array(x))) for i, (x, y) in enumerate(test_set)]
test_df = spark.createDataFrame(test_data, schema).repartition(50)

# Write the DataFrame to Delta Lake format
train_df.write.format("delta").mode("overwrite").option("overwriteSchema", "true").save("./data/mnist_delta/train")
test_df.write.format("delta").mode("overwrite").option("overwriteSchema", "true").save("./data/mnist_delta/test")


Let's reassure ourselves that we can retrieve the images and labels from the Delta Table:

In [ ]:
# Preview one of the images
import matplotlib.pyplot as plt
import numpy as np

# Select one row from the DataFrame
row = train_df.filter(train_df.id == 7).first()

# Extract the image data and label
image_data = row['features']
label = row['label']

# Convert the binary data back to a NumPy array and reshape it
image_array = np.frombuffer(image_data, dtype=np.uint8).reshape(28, 28)

# Plot the image
plt.imshow(image_array, cmap='gray')
plt.title(f'Label: {label}')
plt.show()


## Create a `DataLoader` from Delta Lake Tables
We'll create a PyTorch `DataLoader` that reads data directly from the Delta Lake format using `deltatorch`. `deltatorch` creates an iterable dataset out of the delta tables, then wraps it with a PyTorch DataLoader to enable efficient, parallel data loading into a PyTorch model. This avoids the need to convert the full Delta Lake table to Pandas/NumPy before creating the DataLoader.

In [ ]:
from deltatorch import create_pytorch_dataloader
from deltatorch import FieldSpec
from utils import BinaryToFloatTensor

def create_data_loader(path:str, batch_size:int):
    return create_pytorch_dataloader(
        path,
        id_field="id",
        fields=[
            FieldSpec("features", transform=BinaryToFloatTensor()),
            FieldSpec("label"),
        ],
        num_workers=4,
        shuffle=True,
        batch_size=batch_size,
    )

train_dl = create_data_loader("./data/mnist_delta/train", batch_size=32)

Now we have a `DataLoader`:

In [ ]:
type(train_dl)

Now let's put the `deltatorch` `DataLoader` to work! We'll train a simple CNN on the MNIST data.

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm

# Define the network architecture
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1)
        self.relu = nn.ReLU()
        self.maxpool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc = nn.Linear(32 * 14 * 14, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        x = x.view(-1, 32 * 14 * 14)  # Flatten the tensor
        x = self.fc(x)
        return x

# Create the network, loss function and optimizer
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = SimpleCNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Train the network
for epoch in range(2):  # Loop over the dataset multiple times
    correct = 0
    total = 0
    progress = tqdm(enumerate(train_dl), total=len(train_dl))
    for i, data in progress:
        # Get the inputs; data is a list of [inputs, labels]
        inputs, labels = data['features'], data['label']
        inputs = inputs.unsqueeze(1).to(device)  # Add an extra dimension for the single channel (grayscale)
        labels = labels.to(device)

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward + backward + optimize
        outputs = model(inputs)
        loss = criterion(outputs, labels.long())  # Use long() to ensure the labels are of the correct type
        loss.backward()
        optimizer.step()
        
        # Accuracy
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels.long()).sum().item()

        if (i + 1) % 100 == 0:  # Print training accuracy every 100 batches
            acc = 100 * correct / total
            progress.set_description(f"Loss: {loss.item():.4f}, Accuracy: {acc:.2f}%")

    print(f"Epoch {epoch + 1}, Loss (Train): {loss.item():.4f}, Accuracy (Train): {acc:.2f}%")


Lastly, we'll check the accuracy on the test data.

In [ ]:
test_dl = create_data_loader("./data/mnist_delta/test", batch_size=32)

test_dl = create_data_loader("./data/mnist_delta/test", batch_size=32)

# Function to calculate accuracy
def calculate_accuracy(y_pred, y_test):
    _, predicted = torch.max(y_pred, 1)
    total = y_test.size(0)
    correct = (predicted == y_test).sum().item()
    return correct / total

model.eval()  # Set the model to evaluation mode
total_accuracy = 0

with torch.no_grad():  # We don't need gradients for evaluation
    progress = tqdm(enumerate(test_dl), total=len(test_dl))
    for i, data in progress:
        inputs, labels = data['features'], data['label']
        inputs = inputs.unsqueeze(1).to(device)  # Add an extra dimension for the single channel (grayscale)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(inputs)
        
        # Calculate accuracy
        acc = calculate_accuracy(outputs, labels.long())  # Use long() to ensure the labels are of the correct type
        total_accuracy += acc
        
        #progress.set_description(f"Accuracy: {acc*100:.2f}%")

# Calculate average accuracy over the entire test set
average_accuracy = total_accuracy / len(test_dl)

print(f"Average test accuracy: {average_accuracy*100:.2f}%")

For more examples, see the [examples directory in the deltatorch repository](https://github.com/delta-incubator/deltatorch/tree/main/examples).